In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM, GRU
from keras.preprocessing import text
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from __future__ import print_function
from keras.layers.core import Activation, TimeDistributedDense, RepeatVector
from keras.layers import recurrent
import numpy as np

Using Theano backend.
Using gpu device 0: GeForce GTX 980 Ti (CNMeM is disabled, CuDNN 4007)


In [1]:
class CharacterTable(object):
    def __init__(self, vocab, maxlen):
        self.vocab = vocab
        self.maxlen = maxlen
    
    def encode(self, C, maxlen=None):
        maxlen = maxlen if maxlen else self.maxlen
        X = np.zeros((maxlen, len(self.vocab)))
        for i, c in enumerate(C):
            X[i, c] = 1
        return X
    
    def decode(self, X, calc_argmax=True):
        if calc_argmax:
            X = X.argmax(axis=-1)
        return ','.join(x for x in X)

In [3]:
def generateRandSeq(min, max, len):
    return [np.random.randint(min, max) for _ in range(len)]

In [83]:
TRAINING_SIZE = 10000
TEST_SIZE = 10000
DIGITS = 25
MAXLEN = DIGITS
voc = list(xrange(10))
ctable = CharacterTable(voc, MAXLEN)

In [84]:
inputs = []
outputs = []
inputs_t = []
outputs_t = []
print('Generating data...')
while len(inputs) < TRAINING_SIZE:
    s = generateRandSeq(0, len(voc), DIGITS)
    inputs.append(s)
    # outputs.append(s[::-1])
    outputs.append(sorted(s))

while len(inputs_t) < TEST_SIZE:
    s = generateRandSeq(0, len(voc), DIGITS)
    inputs_t.append(s)
    # outputs_t.append(s[::-1])
    outputs_t.append(sorted(s))
print(inputs[12])
print(outputs[12])

Generating data...
[6, 0, 5, 2, 2, 8, 5, 5, 8, 2, 6, 9, 0, 5, 1, 6, 6, 9, 1, 0, 8, 1, 5, 2, 9]
[0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 5, 5, 5, 5, 5, 6, 6, 6, 6, 8, 8, 8, 9, 9, 9]


In [85]:
print('Vectorization...')
X = np.zeros((len(inputs), MAXLEN, len(voc)), dtype=np.bool)
y = np.zeros((len(outputs), MAXLEN, len(voc)), dtype=np.bool)
for i, sentence in enumerate(inputs):
    X[i] = ctable.encode(sentence, maxlen=MAXLEN)

for i, sentence in enumerate(outputs):
    y[i] = ctable.encode(sentence, maxlen=MAXLEN)

X_test = np.zeros((len(inputs_t), MAXLEN, len(voc)), dtype=np.bool)
y_test = np.zeros((len(outputs_t), MAXLEN, len(voc)), dtype=np.bool)
for i, sentence in enumerate(inputs_t):
    X_test[i] = ctable.encode(sentence, maxlen=MAXLEN)

for i, sentence in enumerate(outputs_t):
    y_test[i] = ctable.encode(sentence, maxlen=MAXLEN)

Vectorization...


In [86]:
print(X.shape)
print(y.shape)
print(X_test.shape)
print(y_test.shape)

(10000, 25, 10)
(10000, 25, 10)
(10000, 25, 10)
(10000, 25, 10)


In [87]:
HIDDEN_SIZE = 128
BATCH_SIZE = 200
LAYERS = 2

In [88]:
print('Build model...')
model = Sequential()
model.add(LSTM(HIDDEN_SIZE, input_shape=(MAXLEN, len(voc)), return_sequences=True))
for _ in range(LAYERS - 2):
    model.add(LSTM(HIDDEN_SIZE, return_sequences=True))

model.add(LSTM(HIDDEN_SIZE))
model.add(RepeatVector(MAXLEN))
for _ in range(LAYERS):
    model.add(LSTM(HIDDEN_SIZE, return_sequences=True))

model.add(TimeDistributedDense(len(voc)))
model.add(Activation('softmax'))

model.compile(optimizer='RMSprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Build model...


In [90]:
early_stopping = EarlyStopping(monitor='val_loss', patience=8)
hist = model.fit(X, y, batch_size=BATCH_SIZE, nb_epoch=161, callbacks=[early_stopping],
          validation_split = 0.1, shuffle=True)

score, acc = model.evaluate(X_test, y_test,
                            batch_size=BATCH_SIZE,
                            show_accuracy=True)
print('Test score:', score)
print('Test accuracy:', acc)

Train on 9000 samples, validate on 1000 samples
Epoch 1/161
9000/9000 [==============================] - 21s - loss: 0.1991 - acc: 0.9171 - val_loss: 0.2455 - val_acc: 0.8904
Epoch 2/161
9000/9000 [==============================] - 22s - loss: 0.2066 - acc: 0.9108 - val_loss: 0.2400 - val_acc: 0.8975
Epoch 3/161
9000/9000 [==============================] - 22s - loss: 0.2026 - acc: 0.9148 - val_loss: 0.4054 - val_acc: 0.8476
Epoch 4/161
9000/9000 [==============================] - 22s - loss: 0.1930 - acc: 0.9354 - val_loss: 0.0587 - val_acc: 0.9975
Epoch 5/161
9000/9000 [==============================] - 21s - loss: 0.1876 - acc: 0.9377 - val_loss: 0.5082 - val_acc: 0.8294
Epoch 6/161
9000/9000 [==============================] - 22s - loss: 0.1961 - acc: 0.9316 - val_loss: 0.0787 - val_acc: 0.9820
Epoch 7/161
9000/9000 [==============================] - 21s - loss: 0.1824 - acc: 0.9392 - val_loss: 0.2359 - val_acc: 0.9222
Epoch 8/161
9000/9000 [==============================] - 21s - 

In [70]:
test = '12345678901234567898'
x_test = ctable.encode(test, maxlen=MAXLEN)
X_test = np.zeros((1, MAXLEN, len(chars)), dtype=np.bool)
X_test[0] = x_test
res = model.predict_classes(X_test)
res[0]

1/1 [==============================] - 0s


array([8, 8, 7, 7, 6, 5, 4, 3, 2, 1, 0, 0, 8, 7, 7, 5, 4, 3, 2, 1])

In [28]:
import matplotlib.pyplot as plt
axes = plt.gca()
x_min = hist.epoch[0]
x_max = hist.epoch[-1]+1
axes.set_xlim([x_min,x_max])

plt.scatter(hist.epoch, hist.history['loss'], color='g')
plt.plot(hist.history['loss'], color='g', label='Training Loss')
plt.scatter(hist.epoch, hist.history['val_loss'], color='b')
plt.plot(hist.history['val_loss'], color='b', label='Validation Loss')
plt.xlabel('epochs')
plt.ylabel('Loss')
plt.title('Training Loss & Validation Loss vs Epochs')
plt.legend()

plt.figure(2)

axes = plt.gca()
x_min = hist.epoch[0]
x_max = hist.epoch[-1]+1
axes.set_xlim([x_min,x_max])

plt.scatter(hist.epoch, hist.history['acc'], color='r')
plt.plot(hist.history['acc'], color='r', label='Training Accuracy')
plt.scatter(hist.epoch, hist.history['val_acc'], color='c')
plt.plot(hist.history['val_acc'], color='c', label='Validation Accuracy')
plt.xlabel('epochs')
plt.ylabel('Accuracy')
plt.title('Trainging Accuracy & Validation Accuracy vs Epochs')
plt.legend()

plt.show()